In [12]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer
import numpy as np
from tensorflow.python.framework import ops
import tflearn
import random

In [13]:
!pip install tflearn

In [14]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [15]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [16]:
import pickle
data=pickle.load(open('training_data','rb'))

In [17]:
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


In [18]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [19]:

# reset underlying graph data
ops.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# load our saved model
model.load('model.tflearn')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2023-02-25 15:45:12.535349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 15:45:12.695728: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled


INFO:tensorflow:Restoring parameters from /Users/aditya/Downloads/Farmer-friendly-contextual-chatbot-master/model.tflearn


In [20]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer().stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [21]:
error_thresh=0.25
def classify(sentence):
     # generate probabilities from the model
    results=model.predict([bow(sentence,words)])[0]
     # filter out predictions below a threshold
    results=[[i,r] for i,r in enumerate(results) if r>error_thresh]
     # sort by strength of probability
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
         # return tuple of intent and probability
    return return_list

In [22]:
context={}
def response(sentence,user_id='123',show_details=False):
    results=classify(sentence)
     # if we have a classification then find the matching intent tag
    if results:
         # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                 # find a tag matching the first result
                    if i['tag'] == results[0][0]:
                         # set context for this intent if necessary
                        if 'context_set' in i:
                            if show_details: print('context',i['context_set'])
                            context[user_id]=i['context_set']
                            
                        # check if this intent is contextual and applies to this user's conversation
                        if not 'context_filter' in i or (user_id in context and 'context_filter' in i and i['context_filter']==context[user_id]):
                            if show_details: print('tag:', i['tag'])
                            # a random response from the intent
                            return random.choice(i['responses'])
            results.pop(0)

In [23]:
response('tell me about wheat protection')


'You can use fumigants like methyl bromide and magnesium or aluminium phosphide'

In [31]:
response('how can i protect my crops')

'Good to see you again'

In [32]:
response('how can i protect my wheat')

'SPRAY OF SOLUBLE NPK  18:18:18 GRADE 10 GM/LTR WATER'

In [33]:
response('how can i save my barley')

'Hello, thanks for visiting'

In [34]:
response('how can i save my cotton')

'Good to see you again'